В рамках учебного проекта сделаны три отдельных решения: первое, для автодополнения слова (LSTM); второе, для автодополнения текста (LSTM); третье, для автодополнения текста (distilgpt2).

Каждое решение было сделано в отдельном .ipynb, в отдельных проектных директориях, с учетом локальных настроек. То, как сделано, можно посмотреть здесь: https://github.com/VitEr-dev/text_autocomplete.git. Описание в README.md. Репозиторий публичный.

В данном файле отдельные .ipynb просто скопированы из исходных трех, последовательно друг за другом, без адаптации и донастроек. Если запустить выполнение здесь - будут ошибки.

Ноутбук (solution_LSTM_word.ipynb). Автодополнение слова, архитектура LSTM

In [ ]:
# Импорт необходимых библиотек
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Импорт необходимых библиотек
import warnings
warnings.filterwarnings('ignore')

# Добавляем путь к src
import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'src'))

# Импорт наших модулей
#import sys
#sys.path.append('./src')

from data_preprocessing import download_and_load_data, preprocess_data, create_sequence_samples
from model import EnhancedLSTMModel
from training import train_with_gradient_clipping, custom_collate_fn
from evaluation import evaluate_model_on_test, test_model_predictions, save_test_results

# Загрузка и предобработка данных
print("Загрузка данных...")
url = "http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip"
tweets_df = download_and_load_data(url)

print(len(tweets_df['text']))

print(f"Сохранено {len(tweets_df['text'])} твитов в raw_dataset.csv")

print("Предобработка данных...")
clean_df, tokenizer = preprocess_data(tweets_df, save_path='./data/')

print(f"После очистки: {len(clean_df)} твитов")


# Создание последовательностей
print("Создание обучающих последовательностей...")
def convert_to_list(input_str):
    if isinstance(input_str, str):
        return [int(x) for x in input_str.strip('[]').split(',') if x.strip()]
    return input_str

clean_df['input_ids'] = clean_df['input_ids'].apply(convert_to_list)

sequence_length = 23
X_data, y_data = create_sequence_samples(clean_df['input_ids'].tolist(), sequence_length)

print(f"Создано {len(X_data)} примеров")
print(f"Форма X: {X_data.shape}, Форма y: {y_data.shape}")
print(f"Пример X: {X_data[0][:5]}... → y: {y_data[0]}")

# Разделение данных
X_train, X_temp, y_train, y_temp = train_test_split(
    X_data, y_data, test_size=0.2, random_state=42, shuffle=True
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True
)

print(f"Размеры выборок:")
print(f"Train: {len(X_train)} примеров")
print(f"Validation: {len(X_val)} примеров")
print(f"Test: {len(X_test)} примеров")

# Сохранение разделенных данных
train_df = pd.DataFrame({'X': X_train.tolist(), 'y': y_train.tolist()})
train_df.to_csv('./data/train_dataset.csv', index=False)

val_df = pd.DataFrame({'X': X_val.tolist(), 'y': y_val.tolist()})
val_df.to_csv('./data/validation_dataset.csv', index=False)

test_df = pd.DataFrame({'X': X_test.tolist(), 'y': y_test.tolist()})
test_df.to_csv('./data/test_dataset.csv', index=False)

# Создание DataLoader'ов
batch_size = 256
pad_token_id = 0

X_train_tensor = torch.tensor(X_train, dtype=torch.long)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True,
    collate_fn=lambda batch: custom_collate_fn(batch, pad_token_id)
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=lambda batch: custom_collate_fn(batch, pad_token_id)
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=lambda batch: custom_collate_fn(batch, pad_token_id)
)

In [ ]:
# Инициализация модели
print("Инициализация модели...")
model = EnhancedLSTMModel(
    vocab_size=30522,
    embedding_dim=128,
    hidden_dim=128,
    num_layers=2,
    pad_token_id=0,
    dropout=0.5,
    use_layer_norm=True,
    use_mean_pooling=False
)

In [ ]:
# Обучение модели
print("Начало обучения...")
model, train_losses, val_losses, learning_rates = train_with_gradient_clipping(
    model, train_loader, val_loader,
    num_epochs=3,
    learning_rate=0.001,
    max_grad_norm=1.0,
    weight_decay=1e-5,
    model_save_path='./models/trained_model_weights_ELSTM_loc1.pth'
)

# Сохранение финальной модели
torch.save(model.state_dict(), './models/final_model.pth')

# Визуализация результатов обучения
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss during Training')

plt.subplot(1, 2, 2)
plt.plot(learning_rates)
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.title('Learning Rate Schedule')
plt.tight_layout()
plt.savefig('./results/training_curves.png')
plt.show()

In [ ]:
import torch
from model import EnhancedLSTMModel

# Тестирование модели
print("Тестирование модели...")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Загрузка лучшей модели для тестирования
best_model = EnhancedLSTMModel(
    vocab_size=30522,
    embedding_dim=128,
    hidden_dim=128,
    num_layers=2,
    pad_token_id=0,
    dropout=0.5,
    use_layer_norm=True,
    use_mean_pooling=False
)
best_model.load_state_dict(torch.load('./models/final_model.pth'))
best_model.to(device)

# Оценка на тестовых данных
metrics, predictions, targets = evaluate_model_on_test(best_model, test_loader, device)

print("\nРЕЗУЛЬТАТЫ ТЕСТИРОВАНИЯ:")
for key, value in metrics.items():
    print(f"{key}: {value}")

# Тестирование конкретных предсказаний
test_model_predictions(best_model, test_dataset, tokenizer, num_examples=20, device=device)

# Сохранение результатов
save_test_results(metrics, predictions, targets, './results/test_results.csv')

print("Все этапы завершены успешно!")


Ноутбук (solution_LSTM_sequence.ipynb). Автодополнение текста, архитектура LSTM

In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import warnings
warnings.filterwarnings('ignore')

import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'src'))

from data_preprocessing_sequence import download_and_load_data, preprocess_data, create_text_sequences
from model_sequence import EnhancedLSTMModel
from training_sequence import train_sequence_model, sequence_collate_fn
from evaluation_sequence import evaluate_sequence_model, test_sequence_predictions
from rouge_score import rouge_scorer


In [ ]:
# Загрузка и предобработка данных
print("Загрузка данных...")
url = "http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip"
tweets_df = download_and_load_data(url)

print("Предобработка данных...")
clean_df, tokenizer = preprocess_data(tweets_df, save_path='./data/')

print("Создание последовательностей для автодополнения...")
clean_df['input_ids'] = clean_df['input_ids'].apply(
    lambda x: [int(i) for i in x.strip('[]').split(',')] if isinstance(x, str) else x
)

# Создаем последовательности: 75% как вход, 25% как цель
X_data, y_data = create_text_sequences(clean_df['input_ids'].tolist(), train_ratio=0.75)

print(f"Создано {len(X_data)} примеров")
print(f"Длина входа: {len(X_data[0])}, длина цели: {len(y_data[0])}")

# Разделение данных
X_train, X_temp, y_train, y_temp = train_test_split(
    X_data, y_data, test_size=0.2, random_state=42, shuffle=True
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True
)

print(f"Размеры выборок:")
print(f"Train: {len(X_train)} примеров")
print(f"Validation: {len(X_val)} примеров")
print(f"Test: {len(X_test)} примеров")

# Сохранение разделенных данных
train_df = pd.DataFrame({'X': X_train, 'y': y_train})
train_df.to_csv('./data/train_sequence_dataset.csv', index=False)

val_df = pd.DataFrame({'X': X_val, 'y': y_val})
val_df.to_csv('./data/validation_sequence_dataset.csv', index=False)

test_df = pd.DataFrame({'X': X_test, 'y': y_test})
test_df.to_csv('./data/test_sequence_dataset.csv', index=False)

# Создание DataLoader'ов
batch_size = 32
pad_token_id = 0

train_loader = DataLoader(
    list(zip(X_train, y_train)),
    batch_size=batch_size,
    shuffle=True,
    collate_fn=lambda batch: sequence_collate_fn(batch, pad_token_id)
)

val_loader = DataLoader(
    list(zip(X_val, y_val)),
    batch_size=batch_size,
    shuffle=False,
    collate_fn=lambda batch: sequence_collate_fn(batch, pad_token_id)
)

test_loader = DataLoader(
    list(zip(X_test, y_test)),
    batch_size=batch_size,
    shuffle=False,
    collate_fn=lambda batch: sequence_collate_fn(batch, pad_token_id)
)


In [ ]:
# Инициализация модели
print("Инициализация модели...")
model = EnhancedLSTMModel(
    vocab_size=30522,
    embedding_dim=128,
    hidden_dim=128,
    num_layers=2,
    pad_token_id=0,
    dropout=0.5,
    use_layer_norm=True,
    use_mean_pooling=False
)

In [ ]:
# Обучение модели
print("Начало обучения...")
model, train_losses, val_losses, train_accuracies, val_accuracies, learning_rates = train_sequence_model(
    model, train_loader, val_loader,
    num_epochs=3,
    learning_rate=0.0005,
    max_grad_norm=1.0,
    weight_decay=1e-4,
    model_save_path='./models/sequence_model.pth'
)

# Визуализация результатов обучения
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss during Training')

plt.subplot(1, 3, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy during Training')

plt.subplot(1, 3, 3)
plt.plot(learning_rates)
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.title('Learning Rate Schedule')
plt.tight_layout()
plt.savefig('./results/sequence_training_curves.png')
plt.show()

In [ ]:
# Сохранение финальной модели
os.makedirs('./models', exist_ok=True)
torch.save(model.state_dict(), './models/final_sequence_model.pth')

In [ ]:
# Проверка параметров сохраненной модели
checkpoint = torch.load('./models/final_sequence_model.pth', map_location='cpu')
print("Ключи в state_dict:")
for key in checkpoint.keys():
    print(f"{key}: {checkpoint[key].shape}")


In [ ]:
# Тестирование модели
print("Тестирование модели...")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
best_model = EnhancedLSTMModel(
    vocab_size=30522,
    embedding_dim=128,
    hidden_dim=128,
    num_layers=2,
    pad_token_id=0,
    dropout=0.5,
    use_layer_norm=True,
    use_mean_pooling=False
)
best_model.load_state_dict(torch.load('./models/final_sequence_model.pth'))
best_model.to(device)

# Оценка с ROUGE метриками
rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)
metrics, all_predictions, all_targets = evaluate_sequence_model(
    best_model, test_loader, tokenizer, rouge_scorer_obj, device
)

print("\nРЕЗУЛЬТАТЫ ТЕСТИРОВАНИЯ:")
for key, value in metrics.items():
    print(f"{key}: {value:.4f}")

# Тестирование конкретных предсказаний
test_sequence_predictions(best_model, list(zip(X_test, y_test)), tokenizer, 
                         num_examples=5, device=device)

print("Все этапы завершены успешно!")

Ноутбук (solution_distilgpt2.ipynb). Автодополнение текста на предобучененной distilgpt2

In [ ]:
# Импорт необходимых библиотек
import warnings
warnings.filterwarnings('ignore')

# Добавляем путь к src
import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'src'))

# Прямой импорт из модулей
from src.model_utils import load_model_and_tokenizer, autocomplete_text
from src.data_utils import prepare_test_data, prepare_test_cases
from src.evaluation_utils import test_autocomplete_performance
from src.evaluation_utils import calculate_rouge_scores
from src.evaluation_utils import print_results_analysis 
from src.evaluation_utils import plot_rouge_scores
from src.evaluation_utils import save_results_to_csv


# Проверка устройства
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")

# КОНФИГУРАЦИЯ ПАРАМЕТРОВ
config = {
    # Параметры модели
    'model_name': "distilgpt2",
    'max_length': 1024,
    'max_new_tokens': 50,
    'temperature': 0.7,
    'top_k': 50,
    'do_sample': True,
    
    # Параметры данных
    'data_path': "./data/dataset_processed.csv",
    'test_size': 0.5,
    'max_samples': 1000,
    'split_ratio': 0.7,
    'min_length': 20,
    'random_state': 42,
    
    # Параметры оценки
    'metrics': ['rouge1', 'rouge2'],
    'use_stemmer': True,
    
    # Параметры сохранения
    'save_results': True,
    'results_dir': "./results",
    'results_file': "autocomplete_results.csv",
    'metrics_file': "autocomplete_metrics.csv",
    'plot_file': "rouge_metrics_autocomplete.png",
    
    # Параметры тестирования
    'num_test_samples': 500
}

def update_config(**kwargs):
    """Обновление конфигурации"""
    config.update(kwargs)
    print("Configuration updated:")
    for key, value in kwargs.items():
        print(f"  {key}: {value}")

def run_complete_testing():
    """Полный pipeline тестирования"""
    print("Starting complete autocomplete testing...")
    print("Current configuration:")
    for key, value in config.items():
        if key not in ['data_path', 'results_dir']:  # Не показываем длинные пути
            print(f"  {key}: {value}")
    
    # 1. Загрузка модели
    model, tokenizer, device = load_model_and_tokenizer(config['model_name'])
    
    # 2. Подготовка данных
    test_texts = prepare_test_data(
        csv_path=config['data_path'],
        test_size=config['test_size'],
        max_samples=config['max_samples'],
        random_state=config['random_state']
    )
    
    test_cases = prepare_test_cases(
        test_texts,
        split_ratio=config['split_ratio'],
        min_length=config['min_length']
    )
    
    # 3. Тестирование
    results, all_predictions, all_references = test_autocomplete_performance(
        model, tokenizer, device, test_cases,
        num_samples=config['num_test_samples'],
        max_length=config['max_length'],
        max_new_tokens=config['max_new_tokens'],
        temperature=config['temperature'],
        top_k=config['top_k'],
        do_sample=config['do_sample']
    )
    
    # 4. Расчет метрик
    rouge_scores = calculate_rouge_scores(
        all_references, all_predictions,
        metrics=config['metrics'],
        use_stemmer=config['use_stemmer']
    )
    
    # 5. Анализ результатов
    print_results_analysis(results, rouge_scores)
    
    # 6. Визуализация
    plot_rouge_scores(
        rouge_scores,
        save_plot=config['save_results'],
        results_dir=config['results_dir'],
        plot_file=config['plot_file']
    )
    
    # 7. Сохранение
    save_results_to_csv(
        results, rouge_scores,
        save_results=config['save_results'],
        results_dir=config['results_dir'],
        results_file=config['results_file'],
        metrics_file=config['metrics_file']
    )
    
    return results, rouge_scores


In [ ]:
# Запуск тестирования с конфигурацией по умолчанию
print("Запуск с конфигурацией по умолчанию:")
results, rouge_scores = run_complete_testing()


In [ ]:
#print("\n" + "="*50)
#print("ИЗМЕНЕНИЕ КОНФИГУРАЦИИ ДЛЯ СЛЕДУЮЩЕГО ЗАПУСКА")
#print("="*50)

update_config(
    max_new_tokens=50,
    temperature=0.7,
    max_samples=2000,
    num_test_samples=1000,
    test_size= 0.5
)

# Запуск с новыми параметрами
print("\nЗапуск с обновленной конфигурацией:")
results, rouge_scores = run_complete_testing()


In [ ]:
# Быстрый запуск без сохранения результатов
update_config(
    max_samples=100,
    num_test_samples=50,
    test_size= 0.5
    save_results=False
)

print("\nБыстрое тестирование:")
results, rouge_scores = run_complete_testing()